# Hands-on Grenoble 2025

In this showcase, we demonstrate:
- how to run axiomatic analyses on existing retrieval models
- axiomatically re-rank retrieval results
- define and use your own custom axiom

## Dependencies


Install the necessary dependencies: `ir_axioms`, `ir_datasets`, and PyTerrier

In [22]:
%pip install "ir_axioms>=1.0" ir_datasets python-terrier

Note: you may need to restart the kernel to use updated packages.


## Data

We will use the MS MARCO and TREC 2019/2020 Deep Learning data using `ir_datasets`.

In [23]:
from ir_datasets import load
from pyterrier.datasets import get_dataset

dataset = load("msmarco-passage/trec-dl-2019")
pt_dataset = get_dataset("irds:msmarco-passage/trec-dl-2019")

Then, index all MSMARCO passages (but skip indexing if index already exists).

In [24]:
from typing import Any
from os.path import exists
from pyterrier.java import required
from pyterrier.terrier import IterDictIndexer

@required
def index_document() -> Any:
    if not exists("./msmarco-index/"):
        indexer = IterDictIndexer("./msmarco-index/")
        # Takes about 10-15 minutes from scratch.
        return indexer.index(pt_dataset.get_corpus_iter())  # type: ignore
    else:
        print("Already indexed.")
        from pyterrier import IndexRef
        return IndexRef.of("./msmarco-index/")  # type: ignore

index = index_document()  # type: ignore


Already indexed.


## Retrieval Models

Next, we define two retrieval models with [PyTerrier](https://pyterrier.readthedocs.io/):
- BM25
- Dirichlet LM


In [27]:
from pyterrier.terrier import Retriever

bm25 = Retriever(
    index, 
    wmodel="BM25",         
    num_results=10
)  # type: ignore
dirichlet = Retriever(
    index, 
    wmodel="Hiemstra_LM", 
    num_results=10
)  # type: ignore

Verify that the retrieval works.

In [28]:
bm25.search("test").head()

,qid,docid,docno,rank,score,query
0,1,8650939,8650939,0,11.597055,test
1,1,5267737,5267737,1,11.410891,test
2,1,708237,708237,2,11.180978,test
3,1,748887,748887,3,11.164428,test
4,1,8175318,8175318,4,11.153779,test


In [29]:
dirichlet.search("test").head()

,qid,docid,docno,rank,score,query
0,1,8650939,8650939,0,6.616294,test
1,1,5267737,5267737,1,6.414387,test
2,1,381232,381232,2,6.166066,test
3,1,7175575,7175575,3,6.096681,test
4,1,708237,708237,4,5.995149,test


## Prepare `ir_axioms` for PyTerrier integration

Let `ir_axioms` infer text and index statistics from the Terrier index.

In [30]:
from ir_axioms.integrations import inject_pyterrier

inject_pyterrier(
    index_location=index,
    dataset=dataset,
)

## Post-hoc Axiomatic Analyses

Similarly to PyTerrier's `Experiment` helper, `ir_axioms` runs axiomatic analyses with the `AxiomaticExperiment` class.

In the constructor parameters, please specify:
- Retrieval systems to test (and their names)
- Axioms to consider
- Maximum ranking depth (top-$k$ when building preference matrices)
- Topics and Qrels
- Index location (needed for IDF-based axioms)
- Document dataset (for full texts)

In [31]:
from ir_axioms.axiom import TFC1, LB1, PROX1, PROX2
from ir_axioms.integrations import AxiomaticExperiment

experiment = AxiomaticExperiment(
    retrieval_systems=[bm25, dirichlet],
    names=["BM25", "DirichletLM"],
    axioms=[
        TFC1(),
        LB1(),
        PROX1(),
        PROX2(),
    ],
    axiom_names=["TFC1", "LB1", "PROX1", "PROX2"],
    depth=5,
    topics=pt_dataset.get_topics()[:50],
    qrels=pt_dataset.get_qrels(),
    filter_by_qrels=True,
    # verbose=True,
)

This will not execute any experiment just yet, but prepares the full configuration.

### Calculate Pairwise Preferences

Output all axiomatic preferences between pairs of documents retrieved in the top-$k$. \
This will output a `DataFrame` with rows representing the document pairs and columns representing the axioms and "pseudo-axioms".

In [32]:
experiment.preferences

Preferences:   0%|          | 0/25 [00:00<?, ?it/s]

Preferences:   0%|          | 0/25 [00:00<?, ?it/s]

Sum term frequencies:   0%|          | 0/5 [00:00<?, ?document/s]

Preferences:   0%|          | 0/25 [00:00<?, ?it/s]

Preferences:   0%|          | 0/25 [00:00<?, ?it/s]

...

Sum term frequencies:   0%|          | 0/5 [00:00<?, ?document/s]

Preferences:   0%|          | 0/25 [00:00<?, ?it/s]

Preferences:   0%|          | 0/25 [00:00<?, ?it/s]

Tokenize documents:   0%|          | 0/5 [00:00<?, ?document/s]

,qid,docid_a,docno_a,rank_a,score_a,query,label_a,iteration_a,docid_b,docno_b,...,score_b,label_b,iteration_b,ORIG_preference,ORACLE_preference,TFC1_preference,LB1_preference,PROX1_preference,PROX2_preference,name
0,156493,8182161,8182161,0,31.156444,do goldfish grow,2,Q0,8182161,8182161,...,31.156444,2,Q0,0.0,0.0,0.0,0.0,0.0,0.0,BM25
1,156493,8182161,8182161,0,31.156444,do goldfish grow,2,Q0,6139386,6139386,...,31.032333,3,Q0,1.0,-1.0,1.0,0.0,1.0,-1.0,BM25
2,156493,8182161,8182161,0,31.156444,do goldfish grow,2,Q0,3288600,3288600,...,30.953843,2,Q0,1.0,0.0,0.0,0.0,0.0,0.0,BM25
3,156493,8182161,8182161,0,31.156444,do goldfish grow,2,Q0,3288596,3288596,...,30.754021,2,Q0,1.0,0.0,0.0,0.0,-1.0,-1.0,BM25
4,156493,8182161,8182161,0,31.156444,do goldfish grow,2,Q0,2259183,2259183,...,30.362477,2,Q0,1.0,0.0,1.0,0.0,0.0,-1.0,BM25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,573724,7104825,7104825,4,15.275695,what are the social determinants of health,2,Q0,394136,394136,...,16.171382,1,Q0,-1.0,1.0,-1.0,0.0,0.0,0.0,DirichletLM
21,573724,7104825,7104825,4,15.275695,what are the social determinants of health,2,Q0,2417985,2417985,...,15.838892,0,Q0,-1.0,1.0,-1.0,0.0,1.0,-1.0,DirichletLM
22,573724,7104825,7104825,4,15.275695,what are the social determinants of health,2,Q0,394139,394139,...,15.709092,1,Q0,-1.0,1.0,0.0,0.0,0.0,0.0,DirichletLM
23,573724,7104825,7104825,4,15.275695,what are the social determinants of health,2,Q0,7104826,7104826,...,15.351193,1,Q0,-1.0,1.0,0.0,0.0,1.0,-1.0,DirichletLM


In [37]:
df = experiment.preferences
df = df[df["LB1_preference"] != 0]
df

,qid,docid_a,docno_a,rank_a,score_a,query,label_a,iteration_a,docid_b,docno_b,...,score_b,label_b,iteration_b,ORIG_preference,ORACLE_preference,TFC1_preference,LB1_preference,PROX1_preference,PROX2_preference,name
4,489204,1479542,1479542,0,38.213874,right pelvic pain causes,2,Q0,973923,973923,...,35.129767,0,Q0,1.0,1.0,-1.0,1.0,0.0,0.0,BM25
9,489204,1051498,1051498,1,37.830403,right pelvic pain causes,2,Q0,973923,973923,...,35.129767,0,Q0,1.0,1.0,-1.0,1.0,0.0,0.0,BM25
14,489204,1950269,1950269,2,36.995804,right pelvic pain causes,0,Q0,973923,973923,...,35.129767,0,Q0,1.0,0.0,-1.0,1.0,0.0,0.0,BM25
19,489204,2268657,2268657,3,36.668262,right pelvic pain causes,0,Q0,973923,973923,...,35.129767,0,Q0,1.0,0.0,-1.0,1.0,0.0,0.0,BM25
20,489204,973923,973923,4,35.129767,right pelvic pain causes,0,Q0,1479542,1479542,...,38.213874,2,Q0,-1.0,-1.0,1.0,-1.0,0.0,0.0,BM25
21,489204,973923,973923,4,35.129767,right pelvic pain causes,0,Q0,1051498,1051498,...,37.830403,2,Q0,-1.0,-1.0,1.0,-1.0,0.0,0.0,BM25
22,489204,973923,973923,4,35.129767,right pelvic pain causes,0,Q0,1950269,1950269,...,36.995804,0,Q0,-1.0,0.0,1.0,-1.0,0.0,0.0,BM25
23,489204,973923,973923,4,35.129767,right pelvic pain causes,0,Q0,2268657,2268657,...,36.668262,0,Q0,-1.0,0.0,1.0,-1.0,0.0,0.0,BM25
1,130510,8612903,8612903,0,23.131254,definition declaratory judgment,3,Q0,7125239,7125239,...,23.093725,1,Q0,1.0,1.0,-1.0,1.0,0.0,0.0,DirichletLM
2,130510,8612903,8612903,0,23.131254,definition declaratory judgment,3,Q0,799647,799647,...,22.860773,3,Q0,1.0,0.0,-1.0,1.0,0.0,0.0,DirichletLM


To view only certain axiom preferences, just select the data frame columns.

In [33]:
experiment.preferences[
    [
        "name",
        "query",
        "docno_a",
        "docno_b",
        "ORIG_preference",
        "ORACLE_preference",
        "TFC1_preference"
    ]
].head()

,name,query,docno_a,docno_b,ORIG_preference,ORACLE_preference,TFC1_preference
0,BM25,do goldfish grow,8182161,8182161,0.0,0.0,0.0
1,BM25,do goldfish grow,8182161,6139386,1.0,-1.0,1.0
2,BM25,do goldfish grow,8182161,3288600,1.0,0.0,0.0
3,BM25,do goldfish grow,8182161,3288596,1.0,0.0,0.0
4,BM25,do goldfish grow,8182161,2259183,1.0,0.0,1.0


### Preference Distribution

Next, we aggregate the axiom preferences and count for each axiom:
- how many times, a "zero" preference is returned (i.e., no preference)
- how many times they agree with ORIG
- how many times they disagree with ORIG

This helps to quickly identify "useful" or "interesting" axioms, e.g., axioms that often disagree with ORIG.


In [34]:
experiment.preference_distribution

,axiom,axiom == 0,axiom == ORIG,axiom != ORIG
0,ORIG,0,120,0
1,ORACLE,54,39,27
2,TFC1,53,51,16
3,LB1,112,8,0
4,PROX1,49,27,44
5,PROX2,26,43,51


In [12]:
experiment.preference_consistency.round(2)

,axiom,ORIG_consistency,ORACLE_consistency,BM25_consistency,DirichletLM_consistency
0,ORIG,1.00,0.78,1.00,1.00
1,ORACLE,0.59,1.00,0.64,0.55
2,TFC1,0.76,0.66,0.68,0.86
3,LB1,1.00,1.00,1.00,1.00
4,PROX1,0.38,0.83,0.45,0.29
5,PROX2,0.46,0.49,0.44,0.48


### Inconsistent Pairs

Looking at inconsistent pairs of documents (i.e., where ORIG contradicts ORACLE and at least one axiom could correct the mistake) helps to understant the problem better.

In [41]:
inconsistent_pairs = experiment.inconsistent_pairs
inconsistent_pairs = inconsistent_pairs[inconsistent_pairs["label_b"] == 0]
inconsistent_pair = inconsistent_pairs.head(10)
inconsistent_pair

,qid,docid_a,docno_a,rank_a,score_a,query,label_a,iteration_a,docid_b,docno_b,...,score_b,label_b,iteration_b,ORIG_preference,ORACLE_preference,TFC1_preference,LB1_preference,PROX1_preference,PROX2_preference,name
16,1110199,4511137,4511137,3,33.556066,what is wifi vs bluetooth,2,Q0,554521,554521,...,34.185628,0,Q0,-1.0,1.0,0.0,0.0,1.0,-1.0,BM25
17,1110199,4511137,4511137,3,33.556066,what is wifi vs bluetooth,2,Q0,398442,398442,...,33.844671,0,Q0,-1.0,1.0,-1.0,0.0,1.0,-1.0,BM25
21,1110199,5218014,5218014,4,32.699297,what is wifi vs bluetooth,2,Q0,554521,554521,...,34.185628,0,Q0,-1.0,1.0,0.0,0.0,-1.0,-1.0,BM25
22,1110199,5218014,5218014,4,32.699297,what is wifi vs bluetooth,2,Q0,398442,398442,...,33.844671,0,Q0,-1.0,1.0,-1.0,0.0,1.0,-1.0,BM25
10,1063750,4337527,4337527,2,29.564491,why did the us volunterilay enter ww1,2,Q0,4788295,4788295,...,30.340022,0,Q0,-1.0,1.0,1.0,0.0,0.0,-1.0,BM25
11,1063750,4337527,4337527,2,29.564491,why did the us volunterilay enter ww1,2,Q0,2997653,2997653,...,29.795791,0,Q0,-1.0,1.0,1.0,0.0,1.0,-1.0,BM25
5,573724,394136,394136,1,30.246689,what are the social determinants of health,1,Q0,2417985,2417985,...,30.607432,0,Q0,-1.0,1.0,0.0,0.0,1.0,-1.0,BM25
10,573724,394139,394139,2,30.145343,what are the social determinants of health,1,Q0,2417985,2417985,...,30.607432,0,Q0,-1.0,1.0,0.0,0.0,1.0,-1.0,BM25
15,573724,3365638,3365638,3,29.125407,what are the social determinants of health,1,Q0,2417985,2417985,...,30.607432,0,Q0,-1.0,1.0,-1.0,0.0,1.0,-1.0,BM25
5,1110199,3838645,3838645,1,19.040404,what is wifi vs bluetooth,2,Q0,398442,398442,...,19.062446,0,Q0,-1.0,1.0,-1.0,0.0,1.0,-1.0,DirichletLM


Using `ir_datasets`, we can also easily get the document texts.

In [39]:
dataset.docs_store().get(inconsistent_pair["docno_a"].iloc[0]).text

'The choice of Bluetooth or WiFi is clear. Bluetooth is a universal standard and easy to use. But, its range is quite limited and sound quality is restricted. WiFi systems have a wider range and can offer excellent fidelity. But, different systems that use WiFi are incompatible, and setup may be difficult.'

In [15]:
inconsistent_pair["label_a"].iloc[0]

2

In [40]:
dataset.docs_store().get(inconsistent_pair["docno_b"].iloc[0]).text

'New Arrival. All kinds of Tablet Pc Wifi Bluetooth Camera are available on LightInTheBox. It is guaranteed that these cool Tablet Pc Wifi Bluetooth Camera are in high quality and affordable price. Check out our website and buy your favorite Tablet Pc Wifi Bluetooth Camera. Also, other kinds of awesome products are available on LightInTheBox.'

In [17]:
inconsistent_pair["label_b"].iloc[0]

0

## Axiomatic Re-Ranking

Now that we have spotted an error, let's fix it by axiomatic re-ranking:
- We first combine multiple axioms in a majority vote
- ... then re-rank based on these majority's preference.

### Majority Vote

Idea: Only change ranking order where multiple axioms agree \
(Fall back to original ranking order if not a majority >= 50%) \
The combined axiom is itself a "pseudo-axiom" implementing the `Axiom` class.

<!-- - combine axioms with Python operators
  `%`: majority vote (with 50% majority)
  `|`: fallback, if preference is 0 -->



In [18]:
from ir_axioms.axiom import ORIG, MajorityVoteAxiom

majority_vote = MajorityVoteAxiom(
    axioms=[
        TFC1(),
        LB1(),
        PROX1(),
        PROX2(),
    ],
    minimum_votes=0.5,
) | ORIG()

### KwikSort

Based on this majority voting, we can apply simple algorithms like KwikSort:
- Similar to quicksort
  1. Select pivot
  1. Compare and split rankings before and after the pivot
  1. Repeat for both sub-rankings
- Results depend on pivot selection!
- Implemented as a PyTerrier `Transformer` module

In [20]:
from ir_axioms.integrations import KwikSortReranker

kwiksort = KwikSortReranker(
    axiom=majority_vote,
    verbose=True,
)
bm25_kwiksort = (bm25 % 10 >> kwiksort) ^ bm25
dirichlet_kwiksort = (dirichlet % 10 >> kwiksort) ^ dirichlet

### Evaluate Re-Rankers

To verify the effectiveness of the axiomatic re-ranking, we run a PyTerrier `Experiment`, comparing the two retrieval systems and their re-ranked variants.

In [21]:
from pyterrier.pipelines import Experiment

Experiment(
    retr_systems=[
        bm25, 
        bm25_kwiksort, 
        dirichlet, 
        dirichlet_kwiksort,
    ],
    names=[
        "BM25",
        "BM25 + KwikSort",
        "DirichletLM",
        "DirichletLM + KwikSort",
    ],
    topics=pt_dataset.get_topics(),
    qrels=pt_dataset.get_qrels(),
    eval_metrics=["ndcg_cut_10"],
)

KwikSort re-rank:   0%|          | 0/200 [00:00<?, ?query/s]

KwikSort re-rank:   0%|          | 0/200 [00:00<?, ?query/s]

,name,ndcg_cut_10
0,BM25,0.479540
1,BM25 + KwikSort,0.484092
2,DirichletLM,0.473787
3,DirichletLM + KwikSort,0.467602


#### Results

Axiomatic re-ranking here only slightly changes the effectiveness.
But: The rankings are now more consistent with axioms.

Potential Improvements:
- Here, we have used a majority voting. But we could also train a classifier for weighting the axioms in another way.
- We have used just a few exemplary axioms, not tailored to the use case.